## Challenge

1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)
import requests

4.8.2


In [2]:
import numpy as np
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [4]:
category = []
title = []
    
date = []
views = []
comments = []    

codimap_explain_list = []
codimap_hashtag_list = []
codimap_imgurl_list = []

page_number = 2

for i in range(1,page_number+1):

    url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page={}'.format(i)
    driver.get(url)
    driver.implicitly_wait(3)
    # 계속 403 오류가 나기 때문에 해더 추가
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'} 
    page = requests.get (url, headers=headers)
    # print(page.status_code)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    style_info = soup.find_all('div', attrs= {'class': 'style-list-information'})
    for i in style_info:
        category.append(i.get_text().replace('\n', ' ').strip().split()[0])  
        title.append(" ".join(i.get_text().replace('\n', ' ').strip().split()[1:]))

    
    post_info = soup.find_all('div', attrs= {'class': 'post-information'})
    # print(post_info[0].get_text())
    for i in post_info:
        date.append(i.get_text().replace('\n', ' ').split()[0])
        views.append(i.get_text().replace('\n', ' ').split()[2])
        comments.append(i.get_text().replace('\n', ' ').split()[-1])

    

    for i in range(1,61): 

        hashtag_str = "" #반복문 돌며 codimap_hashtag_list에 넣어줄 문자열 생성
        
        codi_button = driver.find_element(By.XPATH, '/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{}]'.format(i))  #코디 버튼 클릭
        codi_button.click()
        driver.implicitly_wait(5)

        new_page_source = driver.page_source #이동한 페이지의 페이지 소스 불러오기
        soup2 = BeautifulSoup(new_page_source, 'html.parser')

        codi_info = soup2.find('p',attrs={'class':"styling_txt"}) 
        codi_hashtag = soup2.find_all('a', attrs={"class": "ui-tag-list__item" })
        codi_imgurl = soup2.find('img', attrs={"class": "photo" })

        codimap_explain_list.append(codi_info.get_text()) #리스트에 하나씩 적재
        for i in codi_hashtag:
            hashtag_str += i.get_text() + " " 
        codimap_hashtag_list.append(hashtag_str.strip())
        codimap_imgurl_list.append(codi_imgurl['src'])    

        driver.back()

driver.close()

In [6]:
df1 = pd.DataFrame(zip(category, title, date, views, comments, codimap_explain_list, codimap_hashtag_list, codimap_imgurl_list), columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers','codimap_explain', 'codimap_hashtag', 'codimap_imgurl'])

#데이터타입 변환
df1.views = df1.views.str.replace(",","")
df1.views = df1.views.astype(int)
df1.comment_numbers = df1.comment_numbers.astype(int) 

# df1.info()


In [8]:
# df1
df1.to_csv('0707 양진성_무신사 크롤링 챌린지.csv', encoding = 'utf-8-sig')